In [5]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

import utils_io, utils_misc

from lib_data import DATA_IO

In [6]:
PATH_CURR = os.path.abspath(os.curdir)    # current code
PATH      = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
SUB_LIST  = utils_misc.get_SUB_list(DATA_IO.path_data) # get the SUB id list which we have a recording of them

# 1. Define the LFP Channel MNI Coordinates

In [113]:
# load LFP contact coordinates
MNI_coordinates     = pd.read_csv(DATA_IO.path_coordinates + "contact_coordinates.csv")
MNI_LFP_coordinates = MNI_coordinates[MNI_coordinates.recording_type == "lfp"]
MNI_LFP_channels    = pd.DataFrame(columns=["patient", "hemisphere", "channel", "x", "y", "z"])   

for patient in SUB_LIST:
    
    df_lfp_events   = pd.read_pickle(DATA_IO.path_events +  "SUB_" + patient + "_EVENTS_LFP_RECORDINGS.pkl")
    LFP_hemispheres = df_lfp_events.LFP_hemisphere.unique()
    
    for hemisphere in LFP_hemispheres:
        LFP_channels = df_lfp_events[df_lfp_events.LFP_hemisphere==hemisphere].LFP_channel.unique()

        for channel in LFP_channels:
            
            # get the contact 1 & 2 names
            contact_1 = int(channel.split("-")[0])
            contact_2 = int(channel.split("-")[1])
    
            # get the coordinates of contact 1 & 2
            contact_1_coordinates = MNI_LFP_coordinates[(MNI_LFP_coordinates.patient==int(patient)) & 
                                                        (MNI_LFP_coordinates.hemisphere==hemisphere) & 
                                                        (MNI_LFP_coordinates.contact==contact_1)]
            contact_2_coordinates = MNI_LFP_coordinates[(MNI_LFP_coordinates.patient==int(patient)) & 
                                                        (MNI_LFP_coordinates.hemisphere==hemisphere) & 
                                                        (MNI_LFP_coordinates.contact==contact_2)]
    
            row               = {}
            row["patient"]    = patient
            row["hemisphere"] = hemisphere
            row["channel"]    = channel
            row["x"]          = (float(contact_1_coordinates.x) + float(contact_2_coordinates.x)) / 2
            row["y"]          = (float(contact_1_coordinates.y) + float(contact_2_coordinates.y)) / 2
            row["z"]          = (float(contact_1_coordinates.z) + float(contact_2_coordinates.z)) / 2
            
            MNI_LFP_channels.loc[len(MNI_LFP_channels)] = row 

MNI_LFP_channels.drop_duplicates(inplace=True)
MNI_LFP_channels.to_pickle(DATA_IO.path_coordinates + "MNI_LFP_channels.pkl")

In [28]:
# load dataframe of LFP tapping events
LFP_PSD = utils_io.load_LFP_event_PSD(event_category="tapping", event_laterality="controlateral")

# load LFP contact coordinates
MNI_coordinates     = pd.read_csv(DATA_IO.path_coordinates + "contact_coordinates.csv")
MNI_LFP_coordinates = MNI_coordinates[MNI_coordinates.recording_type == "lfp"]
MNI_LFP_channels    = pd.DataFrame(columns=["patient", "hemisphere", "channel", "x", "y", "z"])                                

for severity in list(LFP_PSD.keys()):
    for patient in LFP_PSD[severity].patient.unique():
    
        # select the hemisphere of the LFP strip for the selected patients
        hemisphere = LFP_PSD[severity][LFP_PSD[severity].patient==patient].LFP_hemisphere.unique()[0]
    
        # find all the rereferenced LFP channels where the controlateral LFP activity is measured for tapping events
        for channel in LFP_PSD[severity][(LFP_PSD[severity].patient==patient) & (LFP_PSD[severity].LFP_hemisphere==hemisphere)].LFP_channel.unique():
    
            # get the contact 1 & 2 names
            contact_1 = int(channel.split("-")[0])
            contact_2 = int(channel.split("-")[1])
    
            # get the coordinates of contact 1 & 2
            contact_1_coordinates = MNI_LFP_coordinates[(MNI_LFP_coordinates.patient==int(patient)) & 
                                                        (MNI_LFP_coordinates.hemisphere==hemisphere) & 
                                                        (MNI_LFP_coordinates.contact==contact_1)]
            contact_2_coordinates = MNI_LFP_coordinates[(MNI_LFP_coordinates.patient==int(patient)) & 
                                                        (MNI_LFP_coordinates.hemisphere==hemisphere) & 
                                                        (MNI_LFP_coordinates.contact==contact_2)]
    
            row               = {}
            row["patient"]    = patient
            row["hemisphere"] = hemisphere
            row["channel"]    = channel
            row["x"]          = (float(contact_1_coordinates.x) + float(contact_2_coordinates.x)) / 2
            row["y"]          = (float(contact_1_coordinates.y) + float(contact_2_coordinates.y)) / 2
            row["z"]          = (float(contact_1_coordinates.z) + float(contact_2_coordinates.z)) / 2
            
            MNI_LFP_channels.loc[len(MNI_LFP_channels)] = row 

MNI_LFP_channels.drop_duplicates(inplace=True)
MNI_LFP_channels.to_pickle(DATA_IO.path_coordinates + "MNI_LFP_channels.pkl")

# 1. DATA IO

In [9]:
# load dataframe of LFP tapping events
LFP_PSD                = utils_io.load_LFP_event_PSD(event_category="tapping", event_laterality="controlateral")
# load LFP channel MNI coordinates
MNI_STN_motor_channels = pd.read_pickle(DATA_IO.path_coordinates + "MNI_LFP_motor_channels.pkl")

feature                   = "event_beta_high_mean"
LFP_feature               = []

for severity in LFP_PSD.keys():
    STN_dynamic               = LFP_PSD[severity][["patient","LFP_hemisphere","LFP_channel",feature]]
    STN_dynamic["hemisphere"] = STN_dynamic.LFP_hemisphere
    STN_dynamic["channel"]    = STN_dynamic.LFP_channel
    STN_dynamic["feature"]    = STN_dynamic[feature]
    STN_dynamic               = STN_dynamic[["patient","hemisphere","channel","feature"]]
    STN_dynamic               = pd.merge(STN_dynamic, MNI_STN_motor_channels, on=['patient', 'hemisphere', 'channel'], how='inner')

    # map left hemisphere to right hemisphere
    STN_dynamic.loc[STN_dynamic.hemisphere == "left", 'x'] *= -1
    STN_dynamic               = STN_dynamic[["patient","feature","x","y","z"]]
    STN_dynamic["severity"]   = severity

    if(len(LFP_feature)==0):
        LFP_feature = STN_dynamic
    else:
        LFP_feature = pd.concat([LFP_feature, STN_dynamic], ignore_index=True)

In [11]:
LFP_feature

,patient,feature,x,y,z,severity
0,009,35.261187,11.79690,-13.02985,-7.939410,noLID_noDOPA
1,009,-1.844241,11.79690,-13.02985,-7.939410,noLID_noDOPA
2,009,-19.183699,11.79690,-13.02985,-7.939410,noLID_noDOPA
3,009,9.062479,11.79690,-13.02985,-7.939410,noLID_noDOPA
4,009,-30.107916,11.79690,-13.02985,-7.939410,noLID_noDOPA
...,...,...,...,...,...,...
4627,110,-15.897942,10.86875,-14.52065,-8.777995,moderate
4628,110,1.089372,12.25470,-13.73770,-6.801190,moderate
4629,110,18.011167,12.25470,-13.73770,-6.801190,moderate
4630,110,-27.121792,11.15885,-13.35355,-6.648490,moderate


In [136]:
data = LFP_feature[LFP_feature.severity=="moderate"]
from statsmodels.formula.api import mixedlm
from sklearn.metrics import mutual_info_score

In [138]:
# Fit a mixed-effects model with 'feature' as the dependent variable,
# 'z' as a continuous predictor, and 'severity' as a fixed effect.
model = mixedlm("feature ~ z * y * x", data, groups=data["patient"])
result = model.fit()

# Print the summary of the model
print(result.summary())

           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  feature   
No. Observations:  592      Method:              REML      
No. Groups:        7        Scale:               2751.5028 
Min. group size:   14       Log-Likelihood:      -3177.2280
Max. group size:   231      Converged:           Yes       
Mean group size:   84.6                                    
-----------------------------------------------------------
           Coef.   Std.Err.   z    P>|z|  [0.025    0.975] 
-----------------------------------------------------------
Intercept 7140.538 2643.564  2.701 0.007 1959.247 12321.828
z          247.743  337.242  0.735 0.463 -413.240   908.726
y          583.141  207.720  2.807 0.005  176.017   990.265
z:y         28.794   25.034  1.150 0.250  -20.271    77.860
x         -521.526  198.230 -2.631 0.009 -910.050  -133.002
z:x        -13.823   25.695 -0.538 0.591  -64.184    36.539
y:x        -42.933   15.641 -2.745 0.006  -73.589  